In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

In [ ]:
def wmape(actual, forecast):
    # we take two series and calculate an output a wmape from it
    # make a series called mape
    se_mape = abs(actual-forecast)/actual

    # get a float of the sum of the actual
    ft_actual_sum = actual.sum()

    # get a series of the multiple of the actual & the mape
    se_actual_prod_mape = actual * se_mape

    # summate the prod of the actual and the mape
    ft_actual_prod_mape_sum = se_actual_prod_mape.sum()

    # float: wmape of forecast
    ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum

    # return a float
    return ft_wmape_forecast

In [ ]:
# Define the function to return the SMAPE value
def smape(actual, predicted):
  
    # Convert actual and predicted to numpy
    # array data type if not already
    if not all([isinstance(actual, np.ndarray), 
                isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual),
        np.array(predicted)
  
    return round(
        np.mean(
            np.abs(predicted - actual) / 
            ((np.abs(predicted) + np.abs(actual))/2)
        )*100, 2
    )

In [ ]:
'''
https://medium.com/mlearning-ai/elasticnet-regression-fundamentals-and-modeling-in-python-8668f3c2e39e
'''
full_data_df = pd.read_csv("result (57).csv")
full_data_df.columns = [c.lower() for c in full_data_df.columns]
orig_full_data_df = full_data_df.copy()
full_data_df.set_index('fscl_wk_end_dt', inplace=True)
full_data_df['ty_avg_unit_price'] = full_data_df.apply(lambda x : x.ty_curr_yr_div_sales_dollars / x.ty_curr_yr_div_sales_units)
full_data_df['ly_1_avg_unit_price'] = full_data_df.apply(lambda x : x.ly_1_curr_yr_div_sales_dollars / x.ly_1_curr_yr_div_sales_units)
train_data_df = full_data_df[full_data_df.khc_fscl_yr_wk_val < 202101]
test_data_df = full_data_df[full_data_df.khc_fscl_yr_wk_val >= 202101]
full_data_df.drop(columns=['uch_level_g', 'basecode', 'khc_fscl_yr_wk_val', 'last_52wk_fscl_wk_bgn_dt_fscl_yr_wk_val', 'last_52wk_fscl_wk_bgn_dt'], inplace=True)
full_data_df = full_data_df.dropna()
train_data_df.drop(columns=['uch_level_g', 'basecode', 'khc_fscl_yr_wk_val', 'last_52wk_fscl_wk_bgn_dt_fscl_yr_wk_val', 'last_52wk_fscl_wk_bgn_dt'], inplace=True)
train_data_df = train_data_df.dropna()
test_data_df.drop(columns=['uch_level_g', 'basecode', 'khc_fscl_yr_wk_val', 'last_52wk_fscl_wk_bgn_dt_fscl_yr_wk_val', 'last_52wk_fscl_wk_bgn_dt'], inplace=True)
test_data_df = test_data_df.dropna()
full_data_df

AttributeError: ignored

In [ ]:
full_data_df.columns

Index(['uch_level_g', 'khc_fscl_yr_wk_val',
       'last_52wk_fscl_wk_bgn_dt_fscl_yr_wk_val', 'last_52wk_fscl_wk_bgn_dt',
       'basecode', 'ty_avg_store_weight_in_div', 'ty_selling_str_cnt_in_div',
       'ty_store_count_in_div', 'ty_div_tdp', 'ty_curr_yr_div_sales_dollars',
       'ty_curr_yr_div_sales_units', 'ly_1_avg_store_weight_in_div',
       'ly_1_selling_str_cnt_in_div', 'ly_1_store_count_in_div',
       'ly_1_div_tdp', 'ly_1_curr_yr_div_sales_dollars',
       'ly_1_curr_yr_div_sales_units', 'ty_avg_r13wk_div_tdp',
       'ly_1_avg_r13wk_div_tdp'],
      dtype='object')

In [ ]:
full_data_df

,ty_avg_store_weight_in_div,ty_selling_str_cnt_in_div,ty_store_count_in_div,ty_div_tdp,ty_curr_yr_div_sales_dollars,ty_curr_yr_div_sales_units,ly_1_avg_store_weight_in_div,ly_1_selling_str_cnt_in_div,ly_1_store_count_in_div,ly_1_div_tdp,ly_1_curr_yr_div_sales_dollars,ly_1_curr_yr_div_sales_units,ty_avg_r13wk_div_tdp,ly_1_avg_r13wk_div_tdp
fscl_wk_end_dt,,,,,,,,,,,,,,
2018-01-20,0.000733,332,1364,0.359968,2239.40,578.0,0.000733,448.0,1364.0,0.489774,3615.71,1070.0000,0.395070,0.489774
2018-01-27,0.000733,423,1364,0.444367,3453.92,883.0,0.000733,386.0,1364.0,0.422290,2665.05,812.0000,0.395127,0.456032
2018-02-03,0.000733,290,1364,0.306313,2120.44,496.0,0.000733,462.0,1364.0,0.501064,3631.95,1074.0000,0.385048,0.471043
2018-02-10,0.000733,617,1364,0.618797,5232.31,2065.0,0.000733,411.0,1364.0,0.446900,3090.83,918.0000,0.404534,0.465007
2018-02-17,0.000733,287,1364,0.290943,2099.08,479.0,0.000733,443.0,1364.0,0.472410,3263.93,952.0000,0.398155,0.466488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-27,0.000800,348,1364,0.369375,2686.70,619.0,0.000733,365.0,1364.0,0.372534,2984.88,697.0000,0.416149,0.447805
2021-12-04,0.000803,389,1364,0.407805,2910.59,658.0,0.000733,667.0,1364.0,0.652781,6384.47,2085.0000,0.421884,0.464778
2021-12-11,0.000791,330,1364,0.339899,2681.52,615.0,0.000733,316.0,1364.0,0.326547,2495.36,576.0000,0.415685,0.454920


In [ ]:
X_train = train_data_df.drop('ty_div_tdp', axis=1).values
y_train = train_data_df[['ty_div_tdp']].values

X_test = test_data_df.drop('ty_div_tdp', axis=1).values
y_test = test_data_df[['ty_div_tdp']].values


In [ ]:
enet_model_1 = ElasticNet().fit(X_train, y_train)
print(enet_model_1.coef_)
print(enet_model_1.intercept_)

[-0.00000000e+00  3.37040719e-04  0.00000000e+00  5.24429645e-05
  1.55857908e-05  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
[0.09540739]


In [ ]:
y_pred_1 = enet_model_1.predict(X_test)

np.sqrt(mean_squared_error(y_test,y_pred_1)), r2_score(y_test,y_pred_1), wmape(y_test,y_pred_1), smape(y_test,y_pred_1)

(0.04140706270400325, 0.8882169643037716, 16.13586189431194, 30.22)

In [ ]:
df_comparison_1 = pd.DataFrame(zip(y_test,y_pred_1), columns=['actual_ty_div_tdp', 'predicted_ty_div_tdp'])
df_comparison_1

,actual_ty_div_tdp,predicted_ty_div_tdp
0,[0.616716893005324],0.611035
1,[0.611073057977664],0.638949
2,[0.385970122053663],0.385674
3,[0.617022239608172],0.641822
4,[0.418892617178531],0.407663
5,[0.644034087303828],0.654110
6,[0.4108429187286521],0.401224
7,[0.6604779309542429],0.689686
8,[0.415606237360925],0.411064
9,[0.642118159467104],0.686756


In [ ]:
enet_model_2 = ElasticNetCV(cv = 10).fit(X_train,y_train)

# If we don't give the lambdas, what's the alpha?

print(enet_model_2.alpha_)
print(enet_model_2.coef_)
print(enet_model_2.intercept_)

0.26167868093110624
[-0.00000000e+00  8.36010776e-04  0.00000000e+00  1.27934915e-05
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.26773597e-06  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
0.032573635481935315


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
y_pred_2 = enet_model_2.predict(X_test)

np.sqrt(mean_squared_error(y_test,y_pred_2)), r2_score(y_test,y_pred_2), wmape(y_test,y_pred_2), smape(y_test,y_pred_2)

(0.017344932887736267, 0.9803857232327289, 15.40589948653973, 29.53)

In [ ]:
df_comparison_2 = pd.DataFrame(zip(y_test,y_pred_2), columns=['actual_ty_div_tdp', 'predicted_ty_div_tdp'])

,actual_ty_div_tdp,predicted_ty_div_tdp
0,[0.616716893005324],0.613633
1,[0.611073057977664],0.624064
2,[0.385970122053663],0.379830
3,[0.617022239608172],0.624142
4,[0.418892617178531],0.402609
5,[0.644034087303828],0.638655
6,[0.4108429187286521],0.401995
7,[0.6604779309542429],0.654784
8,[0.415606237360925],0.407206
9,[0.642118159467104],0.651229


In [ ]:
y_test

array([[0.61671689],
       [0.61107306],
       [0.38597012],
       [0.61702224],
       [0.41889262],
       [0.64403409],
       [0.41084292],
       [0.66047793],
       [0.41560624],
       [0.64211816],
       [0.39917136],
       [0.63664073],
       [0.40877778],
       [0.65345022],
       [0.43467375],
       [0.66332655],
       [0.43134936],
       [0.66034724],
       [0.43962933],
       [0.57184341],
       [0.47120266],
       [0.60783511],
       [0.41930292],
       [0.62867746],
       [0.4298541 ],
       [0.43883264],
       [0.4915329 ],
       [0.65647718],
       [0.33887259],
       [0.49137171],
       [0.19843652],
       [0.28500656],
       [0.20259339],
       [0.43557002],
       [0.28874444],
       [0.33324707],
       [0.42048256],
       [0.6101744 ],
       [0.38055928],
       [0.38684933],
       [0.41307258],
       [0.59388082],
       [0.36470151],
       [0.37109711],
       [0.40153918],
       [0.38206088],
       [0.38289562],
       [0.369

In [ ]:
# define grid
grid = dict()
grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
grid['l1_ratio'] = np.arange(0, 1, 0.01)
# define search
cv_1 = RepeatedKFold(n_splits=10, n_repeats=20, random_state=1)
search_1 = GridSearchCV(enet_model_1, grid, scoring='neg_mean_absolute_error', cv=cv_1, n_jobs=-1)
# perform the search
results_1 = search_1.fit(X_train, y_train)
# summarize
print('MAE: %.3f' % results_1.best_score_)
print('Config: %s' % results_1.best_params_)

MAE: -0.007
Config: {'alpha': 0.0001, 'l1_ratio': 0.0}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.971e-03, tolerance: 1.957e-04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
